设计思路：

假设从某个时间点开始，每周淘汰最后10%。


In [1]:
import pandas as pd
import math
import gc
from IPython.display import display, HTML

import base
import fund_scanner.common_tools.database as db

engine = db.get_sqlalchemy_engine()

In [2]:
# 准备所有参赛者

sql = 'select * from funds a left join funds_update b on a.funds_id=b.funds_id'

df_all_candidates = pd.read_sql(sql, engine).iloc[:,[0,1,2,3,4,5,11]]

df_all_candidates = df_all_candidates.set_index('funds_id')

df_all_candidates['last_price'] = math.nan
df_all_candidates['current_price'] = math.nan
df_all_candidates['gain_ratio'] = math.nan
df_all_candidates['good'] = 0
good = {
    'not_in_yet' : 0,
    'healthy' : 1,
    'hurt_once' : 2,
    'hurt_twice' : 3,
    'out' : 10,
    'noway' : 11
    }
# good值:
#     0 : 未参赛
#     1 : 已参赛，正活跃
#     2 : 受伤 1次
#     3 : 受伤 2次
#     10 : 淘汰
#     11 : 没有参赛资格
# last_price: 上一轮净值
# current_price: 这一轮净值
# gain_ratio: 这一轮涨幅
df_all_candidates['current_ranking'] = 0
df_all_candidates['total_ranking'] = 0

df_all_candidates = df_all_candidates.sort_values('funds_start_date')
df_all_candidates.tail(3)

sql = 'select * from managers_2_funds_history'
df_lookup = pd.read_sql(sql, engine)


sql = 'select * from managers'
df_managers = pd.read_sql(sql, engine)
df_managers = df_managers.set_index('managers_id')
df_managers['current_ranking'] = 0
df_managers['total_ranking'] = 0
display(df_managers.head(3))

,managers_name,managers_code,managers_photo,managers_gendar_guess,working_since_date,funds_history_update_time,current_ranking,total_ranking
managers_id,,,,,,,,
1,艾小军,30277862,None,None,2014-01-13,2017-08-05 03:16:02,0,0
2,薄官辉,30158947,None,None,2015-04-29,2017-08-05 03:18:03,0,0
3,白海峰,30324911,None,None,2016-05-13,2017-08-05 03:20:04,0,0


In [8]:
# game2 游戏规则
# 选择 2016年 前成立的老基金们参赛，从 2016年 开始，每 14天 比一次净值的涨幅，
# 累加每一次比赛得到的排名
# 一直跑到 today，
# 返回所有参赛基金的排名积分的累加

def game2(funds_start_date='2016-1-1', competition_start_date='2016-1-1', competition_end_date='today',
          competition_time_span=14, watching_code=None):

    
    gc.collect()
    # 重新读取数据
    df_competition = df_all_candidates.copy()
    df_managers_competition = df_managers.copy()
    
    if type(watching_code)!=list:
        if watching_code==None:
            watching_code = []
        else:
            watching_code = [watching_code]
    watching_managers_id = df_managers_competition[df_managers_competition['managers_code'].isin(watching_code)].index.tolist()
    
    
    # 所有队伍先设置状态
    df_competition.loc[:,'good'] = good['healthy']
    
    # 2017年1月1日以后成立的基金或者没有写成立时间的基金没有参赛资格
    df_competition.loc[df_competition['funds_start_date']>pd.to_datetime(funds_start_date), 'good'] = good['noway']
    df_competition.loc[df_competition['funds_start_date'].isnull(),'good'] = good['noway']

    #资产规模小于1亿，或者没有数据的没有资格参赛
    df_competition.loc[df_competition['funds_amount']<1, 'good'] = good['noway']
    df_competition.loc[df_competition['funds_amount'].isnull(),'good'] = good['noway']

    df_competition = df_competition[df_competition['good']==good['healthy']]
    #参赛队伍准备完毕

    
    #比赛从最早一直基金成立开始
    #start_point = pd.to_datetime(df_competition.iloc[0,3])
    #比赛从2013年1月1日开始
    start_point = pd.to_datetime(competition_start_date)
    round_count = 1
    current_date = start_point
    while current_date < pd.to_datetime(competition_end_date):
        df_current_price = pd.read_sql('select * from funds_historical_price where funds_price_date=\'%s\''%current_date, engine)
        if len(df_current_price)<10:
            current_date = current_date + pd.DateOffset(1)
            continue

        df_lookup_current_date = df_lookup[(df_lookup['history_start_date']<current_date) & (df_lookup['history_end_date']>current_date)]
        
        # 把当前价格设为上一期价格
        df_competition['last_price'] = df_competition['current_price']
        # 设置当天价格（如果当天没几个价格就查次日的）
        df_current_price = df_current_price.set_index('funds_id')
        df_competition['current_price'].update(df_current_price['funds_price_adjust'])

        # 如果价格从非0到有，则说明价格变化了，计算变化率
        df_competition['gain_ratio'] = \
        ( df_competition['current_price'] - df_competition['last_price'] ) / df_competition['last_price']

        # 按变化率排序，累加排名
        df_competition_round = df_competition[df_competition['gain_ratio']>-100].sort_values('gain_ratio', ascending=True)
        df_competition_round = df_competition_round.assign(current_ranking=[i for i in range(len(df_competition_round))])
        df_competition_round['current_ranking'] = df_competition_round['current_ranking'] - len(df_competition_round)/2
        #display(df_competition_round.sort_values('current_ranking', ascending=False))

        df_competition.loc[df_competition['gain_ratio']>-100, 'total_ranking'] = df_competition['total_ranking'] + df_competition['current_ranking']

        df1 = df_competition_round.merge(df_lookup_current_date, left_index=True, right_on='funds_id')
        #display(df1)
        df = df1.loc[:,['managers_id', 'current_ranking']]
        df = df.groupby(['managers_id']).mean()
        df_managers_competition['current_ranking']=0
        df_managers_competition.update(df)
        df_managers_competition['total_ranking'] = df_managers_competition['total_ranking'] + df_managers_competition['current_ranking']
        
        print('Round %d: %s'% (round_count, current_date))

        round_count += 1
        current_date = current_date + pd.DateOffset(competition_time_span)
        
        # 观测特定经理的状态
        if len(watching_code)>0:
            display(df_managers_competition.loc[df_managers_competition['managers_code'].isin(watching_code), ['managers_name','managers_code','current_ranking','total_ranking']].sort_index())
            display(df1.loc[df1['managers_id'].isin(watching_managers_id), ['funds_code','funds_name_full','funds_type','last_price','current_price','gain_ratio','current_ranking','managers_id']].sort_index())


    # The winner is (return the full list, and the winner is on the top with highest ranking):
    return df_competition.loc[df_competition['good']==good['healthy']].sort_values('total_ranking', ascending=False),\
            df_managers_competition.sort_values('total_ranking', ascending=False)

#df1,df2 = game2(watching_code=['30142280','30422170','30285112'], competition_start_date='2017-2-1',competition_end_date='2017-3-1', competition_time_span=7)
df1,df2 = game2(competition_start_date='2017-07-01', competition_end_date='today', competition_time_span=3)

Round 1: 2017-07-02 00:00:00
Round 2: 2017-07-05 00:00:00
Round 3: 2017-07-08 00:00:00
Round 4: 2017-07-11 00:00:00
Round 5: 2017-07-14 00:00:00
Round 6: 2017-07-17 00:00:00
Round 7: 2017-07-20 00:00:00
Round 8: 2017-07-23 00:00:00
Round 9: 2017-07-26 00:00:00
Round 10: 2017-07-29 00:00:00
Round 11: 2017-08-01 00:00:00
Round 12: 2017-08-04 00:00:00


In [9]:
display(df2.head(20))
display(df2.tail(5))


,managers_name,managers_code,managers_photo,managers_gendar_guess,working_since_date,funds_history_update_time,current_ranking,total_ranking
managers_id,,,,,,,,
279,官泽帆,30366472,None,None,2016-09-24,2017-08-05 04:03:03,1062.5,8105.000000
1378,周寒颖,30466640,None,None,2016-06-03,2017-08-05 06:37:03,915.5,8019.000000
717,马君,30185328,None,None,2012-09-04,2017-08-05 10:15:03,991.5,7537.666667
485,刘荟,30142052,None,None,2016-01-04,2017-08-05 01:56:04,959.5,7210.000000
429,孔祥鹏,30130842,None,None,2017-06-28,2017-08-05 09:09:03,1053.5,6730.000000
419,蒋一茜,30383521,None,None,2015-10-24,2017-08-05 08:49:03,486.5,6575.000000
1384,张军,30043326,None,None,2008-03-08,2017-08-05 06:49:03,783.0,6413.500000
211,方磊,30165669,None,None,2012-08-01,2017-08-05 10:27:02,1052.5,6051.000000
1362,占冠良,30038356,None,None,2013-04-14,2017-08-05 06:05:03,1004.5,5872.000000


,managers_name,managers_code,managers_photo,managers_gendar_guess,working_since_date,funds_history_update_time,current_ranking,total_ranking
managers_id,,,,,,,,
1328,赵蓓,30301680,None,None,2014-11-18,2017-08-05 04:57:04,-804.5,-6741.5
904,谭冬寒,30487957,None,None,2016-09-02,2017-08-05 07:53:03,-662.5,-6935.0
431,蒯学章,30081306,None,None,2016-11-24,2017-08-05 09:13:03,-1009.5,-7057.0
1409,郑磊,30119501,None,None,2014-12-17,2017-08-05 07:40:03,-1034.5,-7146.0
34,陈斌,30322659,None,None,2015-02-02,2017-08-05 04:25:03,-1041.0,-7843.5


In [10]:
df2[df2['managers_name']=='桂征辉']

,managers_name,managers_code,managers_photo,managers_gendar_guess,working_since_date,funds_history_update_time,current_ranking,total_ranking
managers_id,,,,,,,,
282,桂征辉,30360645,None,None,2015-07-21,2017-08-05 04:09:03,12.5,-7.666667
